In [7]:
import io
import struct
from configs import constants

class Reader:
    reader: io.BufferedReader
    data: bytes
    buffer_value : any 

    def __init__(self, reader: io.BufferedReader = None):
        if reader is None:
            raise ValueError("Reader: None reader given")
        self.reader = reader
        self.data = b""

    def read(self,size:int = None):
        if self.reader is None:
            raise ValueError("Plz init the reader jeezzz")
        
        self.data = self.reader.read(size)
        return self.data 

    def read_int(self):
        return int.from_bytes(self.read(size = constants.LenInt32 ))
    
    def read_byte(self):
        return self.read(size = constants.LenInt32 )

    def read_tlv(self):
        buffer = io.BytesIO()
        datatype = self.read_byte()
        buffer.write(datatype)

        length = self.read_int()
        buffer.write(length.to_bytes(constants.LenInt32))

        value = self.read(length)
        buffer.write(value)
        self.buffer_value =  buffer.getvalue()
        return self.buffer_value




In [8]:
stream = io.BytesIO(b'\x01\x00\x00\x00\xff\xff\xff\xff')

In [2]:
stream = io.BytesIO(b'\x01\x00\x00\x00\xff\xff\xff\xffHello!')

r = Reader(stream)



In [3]:
r.read()

b'\x01\x00\x00\x00\xff\xff\xff\xffHello!'

In [9]:
from marshaller.valmarshal import TLVMarshaler
tlv = TLVMarshaler(value = "Shreyank")

In [10]:
tlv.tlv_marshal()

Encoded TLV: [2, 8, 0, 0, 0, 83, 104, 114, 101, 121, 97, 110, 107] for value Shreyank


b'\x02\x08\x00\x00\x00Shreyank'

In [11]:
stream = io.BytesIO(tlv.tlv)
r= Reader(stream)
r.read_tlv()

b'\x02\x08\x00\x00\x00Shreyank'

In [12]:
binary_data = b'\x02\x04\x00\x00\x00demo'
stream = io.BytesIO(binary_data)
r = Reader(stream)
r.read_tlv()

b'\x02\x04\x00\x00\x00demo'

In [1]:
from modals.db import DataBase
import modals

db = DataBase("t1db")
columns_map = {"id": modals.columns.Column("id", 1, False)}  # Assume
columns_list = ["id"]
table = db.create_tables("mytable", columns_list, columns_map)

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
The value b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 30 bytes with total data being 30


In [4]:
t = db.tables['mytable']

In [6]:
c_obj = t.columns[t.columnNames[-1]]

In [9]:
m = c_obj.column_marshaler

In [11]:
m.print_bytes()

the byte value is [99, 0, 0, 0, 18, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0]


In [12]:
from marshaller.colmarshal import ColumnUnmarshaler
un = ColumnUnmarshaler(m.column_tlv_encoded)

In [14]:
m.column_tlv_encoded

b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00'

In [15]:
import marshaller
import marshaller.colmarshal

m = marshaller.colmarshal.ColumnMarshaler(name = "id",datatype=1,allowNull=False)

In [16]:
m.marshal_column()

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False


b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00'

In [18]:
m.print_bytes()

the byte value is [99, 0, 0, 0, 18, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0]


In [19]:
u = marshaller.colmarshal.ColumnUnmarshaler(m.column_tlv_encoded)

In [35]:
u = ColumnUnmarshaler(m.column_tlv_encoded)
c = u.unmarshal_column()

Outer type is  99
Column length is  18
Decoded TLV: type=2, length=2, value=id
Name field decoded at offset 15
Decoded TLV: type=5, length=4, value=1
Datatype field decoded at offset 24
Decoded TLV: type=4, length=1, value=False
AllowNull field decoded at offset 30


In [37]:
c.name,c.allow_null,c.datatype

('id', False, 1)

In [2]:
import modals.columns
from modals.db import DataBase
import modals

db = DataBase("t3db")

Path already exists!


In [2]:
id = table.columns['id']

In [3]:
name = table.columns['name']

In [4]:
mid = id.column_marshaler.marshal_column()

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False


In [5]:
type(mid)

bytes

In [6]:
import marshaller
import marshaller.colmarshal
u = marshaller.colmarshal.ColumnUnmarshaler(mid)

In [7]:
u.unmarshal_column()
u.print_col()

Outer type is  99
Column length is  18
Decoded TLV: type=2, length=2, value=id
Name field decoded at offset 15
Decoded TLV: type=5, length=4, value=1
Datatype field decoded at offset 24
Decoded TLV: type=4, length=1, value=False
AllowNull field decoded at offset 30
the column name is id with the dtype 1 and has allownull False


In [8]:
name = name.column_marshaler.marshal_column()

Encoded TLV: [2, 4, 0, 0, 0, 110, 97, 109, 101] for value name
Encoded TLV: [5, 4, 0, 0, 0, 2, 0, 0, 0] for value 2
Encoded TLV: [4, 1, 0, 0, 0, 1] for value True


In [10]:
u = marshaller.colmarshal.ColumnUnmarshaler(name)

In [16]:
u.unmarshal_column()

error: unpack requires a buffer of 4 bytes

In [12]:
u.print_col()

the column name is name with the dtype 2 and has allownull True


In [13]:
with open("/home/shreyank/db/database_shrey/data/t3db/mytable.bin", "rb") as f:
    data = f.read()


In [14]:
data

b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00c\x00\x00\x00\x13\x00\x00\x00\x02\x04\x00\x00\x00name\x05\x04\x00\x00\x00\x02\x00\x00\x00\x04\x01\x00\x00\x00\x01'

In [15]:
list(data)

[99,
 0,
 0,
 0,
 18,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 105,
 100,
 5,
 4,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 4,
 1,
 0,
 0,
 0,
 0,
 99,
 0,
 0,
 0,
 19,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 110,
 97,
 109,
 101,
 5,
 4,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 4,
 1,
 0,
 0,
 0,
 1]

Path already exists!


AttributeError: 'DataBase' object has no attribute 'read_tables'

In [ ]:
from configs import constants
from typing import List
import marshaller
import marshaller.colmarshal
import marshaller.valmarshal
import modals
import io 
from typing import Any,Annotated
import re

import modals.columndefreader
import modals.columndefwriter
import modals.columns
import modals.reader 

import traceback

import modals.recordparser



class Table:
    name: str
    file: io.TextIOWrapper | io.BufferedReader | Any  
    columnNames: Annotated[list[str],"has List of string where each of them is the column name "] # list of the column names
    columns: Annotated[dict[str, modals.columns.Column],"Map or dict which has column name as key and value is the column object itself "]  # map[column name] = modals.column object
    reader: modals.reader.Reader          #  modals.reader class
    recordParser:  modals.recordparser.RecordParser
    columnDefReader: modals.columndefreader.ColumnDefReader
    wal: Any
    _file_path:str 

    def __init__(self, file_path= None , reader= None  , wal= None ,columns = None , columnNames= None,recordParser= None  ):
        ''''Dont pass col def reader coz its done when we pass reader'''
        self._file_path = file_path
        self.file = open(file_path, "r+b")
        self.name = self.getTableName()
        self.columns = {}
        self.reader = reader
        self.columnDefReader = modals.columndefreader.ColumnDefReader(self.reader)
        self.wal = wal
        self.columns = columns 
        self.columnNames = columnNames # list of string 
        self.recordParser = modals.recordparser.RecordParser(file = self.file , reader = self.reader,columns=self.columnNames)

    
    def getTableName(self) -> str:
        # Get filename attribute safely
        filename = getattr(self.file, "name", getattr(self.file, "filename", "unknown"))
        # Split on both underscores and spaces
        parts = re.split(r'[_ ]+', filename)
        # Take first chunk, remove extension
        base = parts[0].split('.')[0]
        return base.split("/")[-1]
    
    def getColumnNames(self):
        return self.columnNames
    
    def WriteColumnDefinition(self):
        for col in self.columnNames:
            coltlv = self.columns[col].column_marshaler.marshal_column()
            colwriter = modals.columndefwriter.ColumnDefinitionWriter(self.file).write(coltlv)
            print(f"The value {coltlv} was wrote to the colwriter of length {colwriter} bytes with total data being {len(coltlv)}" )


    def ReadColumnDefinitions(self):
        self.columnNames = []
        self.columns = {}
        buffer = bytearray(1024 * 20)  # Larger for full col defs
        print("cmonnnnn mannnn")
        while True:
            
            try:
                buffer[:] = b'\x00' * len(buffer)
                length_read = self.columnDefReader.read(buffer)
                if length_read ==0:  
                    break
                print(buffer)
                self.buffer = buffer
                col_unmarshaler = marshaller.colmarshal.ColumnUnmarshaler(buffer[:length_read])
                col = col_unmarshaler.unmarshal_column()
                self.columnNames.append(col.name)
                self.columns[col.name] = col
                print(f"Loaded column: {col.name} (type: {constants.typemap.get(col.datatype, 'unknown')}, null: {col.allow_null})")
            except Exception as e:
                print(e)
                traceback.print_exc()
                return
                    
            print(f"Total columns loaded: {len(self.columnNames)}")




    def print_table(self):
        print(f"the table has the name {self.name} with columns {self.columnNames} and file path mm {self._file_path}")

        
    def validate_columns(self,columns):
        '''expects column name : value kind of dictionary'''
        for col,value in columns:
            try:
                self.columns[col]
            except KeyError as e:
                print("No such column found")
            
            if self.columns[col].allow_null==False and value ==None:
                raise IndexError("Column value cant be null")
        return None
    

    def seek_until(self,targetType):
        while(True):
            dtype = self.reader.read_byte()
            if dtype==targetType:
                self.file.seek(-1*constants.LenByte,io.SEEK_CUR)
                return
            length = self.reader.read_int()
            self.file.seek(length,io.SEEK_CUR)



    def ensureFilePointer(self):
        self.file.seek(0,io.SEEK_SET)
        self.seek_until(constants.TypeRECORD)
        return 



    def insert(self,record):
        '''record be dictionary'''
        self.file.seek(0,2) # move the file pointer to the end
        sizeOfRecord = 0
        for col in self.columnNames:  # maybe make primary key and stuff
            if record.get(col) is None:
                raise ValueError(f"The record doesnt have value for the given column {col}")
            val = record[col]
            tlv = marshaller.valmarshal.TLVMarshaler(val)
            tlv.tlv_marshal()
            length = tlv.tlv_length()
            sizeOfRecord+=length
        
        buffer = io.BytesIO()
        typebuffer = tlv.marshal(constants.TypeRECORD)
        buffer.write(typebuffer)

        size_buffer = tlv.marshal(sizeOfRecord)
        buffer.write(size_buffer)

        for col in self.columnNames:
            val= record[col]
            tlv_record = tlv.tlv_marshal(val)
            buffer.write(tlv_record)

        n = self.file.write(buffer.getvalue())

        if n==len(buffer.getvalue()):
            print("Record registered successfullly to the table")
        else:
            raise Exception("Couldnt write to the table")

    
    def validateWhereStatement(self,wherestmt):
        for col,val in wherestmt.items():
            if col not in self.columns:
                raise ValueError("UNkwon column in the where statement")
        return 

    def ensureColumnLength(self,record):
        if len(record.keys())!=len(self.columnNames):
            raise ValueError("Column lengths mismatch couldnt retrieve all the column from the table")

    def evaluateWhereStatement(self,wherestmt,record):
        for key,value in wherestmt.items():
            if record[key]!=value:
                return False
        return True

    def select(self,wherestmt):
        self.ensureFilePointer()
        self.validateWhereStatement(wherestmt)
        results = []
        while(True):
            raw = self.recordParser.parse()
            if raw is None:
                return # end of the file signal
            rawrecord = self.recordParser.value
            self.ensureColumnLength()
            if not self.evaluateWhereStatement(wherestmt,rawrecord.record):
                continue
            results.append(rawrecord.record)







# from Writing Column Definitions is remaining 

class DeletetableRecord :
    offset : int 
    length  : int 

    def __init__(self,offset= None  , length= None ):
        self.offset = offset
        self.length = length
        


In [ ]:
from configs import constants
from typing import List
import marshaller
import marshaller.colmarshal
import marshaller.valmarshal
import modals
import io 
from typing import Any,Annotated
import re

import modals.columndefreader
import modals.columndefwriter
import modals.columns
import modals.reader 

import traceback

import modals.recordparser



class Table:
    name: str
    file: io.TextIOWrapper | io.BufferedReader | Any  
    columnNames: Annotated[list[str],"has List of string where each of them is the column name "] # list of the column names
    columns: Annotated[dict[str, modals.columns.Column],"Map or dict which has column name as key and value is the column object itself "]  # map[column name] = modals.column object
    reader: modals.reader.Reader          #  modals.reader class
    recordParser:  modals.recordparser.RecordParser
    columnDefReader: modals.columndefreader.ColumnDefReader
    wal: Any
    _file_path:str 

    def __init__(self, file_path= None , reader= None  , wal= None ,columns = None , columnNames= None,recordParser= None  ):
        ''''Dont pass col def reader coz its done when we pass reader'''
        self._file_path = file_path
        self.file = open(file_path, "r+b")
        self.name = self.getTableName()
        self.columns = {}
        self.reader = reader
        self.columnDefReader = modals.columndefreader.ColumnDefReader(self.reader)
        self.wal = wal
        self.columns = columns 
        self.columnNames = columnNames # list of string 
        self.recordParser = modals.recordparser.RecordParser(file = self.file , reader = self.reader,columns=self.columnNames)

    
    def getTableName(self) -> str:
        # Get filename attribute safely
        filename = getattr(self.file, "name", getattr(self.file, "filename", "unknown"))
        # Split on both underscores and spaces
        parts = re.split(r'[_ ]+', filename)
        # Take first chunk, remove extension
        base = parts[0].split('.')[0]
        return base.split("/")[-1]
    
    def getColumnNames(self):
        return self.columnNames
    
    def WriteColumnDefinition(self):
        for col in self.columnNames:
            coltlv = self.columns[col].column_marshaler.marshal_column()
            colwriter = modals.columndefwriter.ColumnDefinitionWriter(self.file).write(coltlv)
            print(f"The value {coltlv} was wrote to the colwriter of length {colwriter} bytes with total data being {len(coltlv)}" )


    def ReadColumnDefinitions(self):
        self.columnNames = []
        self.columns = {}
        buffer = bytearray(1024 * 20)  # Larger for full col defs
        print("cmonnnnn mannnn")
        while True:
            
            try:
                buffer[:] = b'\x00' * len(buffer)
                length_read = self.columnDefReader.read(buffer)
                if length_read ==0:  
                    break
                print(buffer)
                self.buffer = buffer
                col_unmarshaler = marshaller.colmarshal.ColumnUnmarshaler(buffer[:length_read])
                col = col_unmarshaler.unmarshal_column()
                self.columnNames.append(col.name)
                self.columns[col.name] = col
                print(f"Loaded column: {col.name} (type: {constants.typemap.get(col.datatype, 'unknown')}, null: {col.allow_null})")
            except Exception as e:
                print(e)
                traceback.print_exc()
                return
                    
            print(f"Total columns loaded: {len(self.columnNames)}")




    def print_table(self):
        print(f"the table has the name {self.name} with columns {self.columnNames} and file path mm {self._file_path}")

        
    def validate_columns(self,columns):
        '''expects column name : value kind of dictionary'''
        for col,value in columns:
            try:
                self.columns[col]
            except KeyError as e:
                print("No such column found")
            
            if self.columns[col].allow_null==False and value ==None:
                raise IndexError("Column value cant be null")
        return None
    

    def seek_until(self,targetType):
        while(True):
            dtype = self.reader.read_byte()
            if dtype==targetType:
                self.file.seek(-1*constants.LenByte,io.SEEK_CUR)
                return
            length = self.reader.read_int()
            self.file.seek(length,io.SEEK_CUR)



    def ensureFilePointer(self):
        self.file.seek(0,io.SEEK_SET)
        self.seek_until(constants.TypeRECORD)
        return 



    def insert(self,record):
        '''record be dictionary'''
        self.file.seek(0,2) # move the file pointer to the end
        sizeOfRecord = 0
        for col in self.columnNames:  # maybe make primary key and stuff
            if record.get(col) is None:
                raise ValueError(f"The record doesnt have value for the given column {col}")
            val = record[col]
            tlv = marshaller.valmarshal.TLVMarshaler(val)
            tlv.tlv_marshal()
            length = tlv.tlv_length()
            sizeOfRecord+=length
        
        buffer = io.BytesIO()
        typebuffer = tlv.marshal(constants.TypeRECORD)
        buffer.write(typebuffer)

        size_buffer = tlv.marshal(sizeOfRecord)
        buffer.write(size_buffer)

        for col in self.columnNames:
            val= record[col]
            tlv_record = tlv.tlv_marshal(val)
            buffer.write(tlv_record)

        n = self.file.write(buffer.getvalue())

        if n==len(buffer.getvalue()):
            print("Record registered successfullly to the table")
        else:
            raise Exception("Couldnt write to the table")

    
    def validateWhereStatement(self,wherestmt):
        for col,val in wherestmt.items():
            if col not in self.columns:
                raise ValueError("UNkwon column in the where statement")
        return 

    def ensureColumnLength(self,record):
        if len(record.keys())!=len(self.columnNames):
            raise ValueError("Column lengths mismatch couldnt retrieve all the column from the table")

    def evaluateWhereStatement(self,wherestmt,record):
        for key,value in wherestmt.items():
            if record[key]!=value:
                return False
        return True

    def select(self,wherestmt):
        self.ensureFilePointer()
        self.validateWhereStatement(wherestmt)
        results = []
        while(True):
            raw = self.recordParser.parse()
            if raw is None:
                return # end of the file signal
            rawrecord = self.recordParser.value
            self.ensureColumnLength()
            if not self.evaluateWhereStatement(wherestmt,rawrecord.record):
                continue
            results.append(rawrecord.record)







# from Writing Column Definitions is remaining 

class DeletetableRecord :
    offset : int 
    length  : int 

    def __init__(self,offset= None  , length= None ):
        self.offset = offset
        self.length = length
        


In [25]:
import modals.db
db = modals.db.DataBase("dxcvb7f")
columns_map = {"id": modals.columns.Column("id", 1, False),"name":modals.columns.Column("name", 2, False)} 
columns_list = ["id","name"]
table = db.create_tables("mytable4", columns_list, columns_map)

total tables in the db are []
Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
Column TLV: [99, 0, 0, 0, 22, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0] (len=30)
The value b'c\x00\x00\x00\x16\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 30 bytes with total data being 30
Encoded TLV: [2, 4, 0, 0, 0, 110, 97, 109, 101] for value name
Encoded TLV: [5, 4, 0, 0, 0, 2, 0, 0, 0] for value 2
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
Column TLV: [99, 0, 0, 0, 24, 0, 0, 0, 2, 4, 0, 0, 0, 110, 97, 109, 101, 5, 4, 0, 0, 0, 2, 0, 0, 0, 4, 1, 0, 0, 0, 0] (len=32)
The value b'c\x00\x00\x00\x18\x00\x00\x00\x02\x04\x00\x00\x00name\x05\x04\x00\x00\x00\x02\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 32 bytes with total data being 32
cmonnnnn mannn

In [26]:
db.read_tables()

cmonnnnn mannnn
bytearray(b'c\x00\x00\x00\x16\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'mytable4': <modals.tables.Table at 0x7a7af9552e70>}

In [27]:
t = db.tables

In [28]:
t.keys()

dict_keys(['mytable4'])

In [29]:
mytable = t['mytable4']

In [30]:
mytable.print_table()

the table has the name mytable4 with columns ['id', 'name'] and file path mm data/dxcvb7f/mytable4.bin


In [31]:
mytable.print_table()

the table has the name mytable4 with columns ['id', 'name'] and file path mm data/dxcvb7f/mytable4.bin


In [32]:
mytable.insert({
    "id":1,
    "name":"Shreyank"
})

Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [2, 8, 0, 0, 0, 83, 104, 114, 101, 121, 97, 110, 107] for value Shreyank
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [2, 8, 0, 0, 0, 83, 104, 114, 101, 121, 97, 110, 107] for value Shreyank
Record registered successfullly to the table


In [4]:
from modals.columns import Column
from modals.db import DataBase
import modals
db = DataBase("db")  # Creates ./data/db/mytable.bin
table = db.create_tables("mytable", ["id", "name"], {
    "id": modals.columns.Column("id", 1, False),
    "name": modals.columns.Column("name", 2, False)
})
table.insert({"id": 1, "name": "Alice"})
table.insert({"id": 2, "name": "Bob"})
print(table.select({"id": 1}))  # [{'id': 1, 'name': 'Alice'}]
print(table.select({}))  # Both
table.print_table()  # Columns intact          # → all records

Path already exists!
total tables in the db are []
Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
Column TLV: [99, 0, 0, 0, 22, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0] (len=30)
The value b'c\x00\x00\x00\x16\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 30 bytes with total data being 30
Encoded TLV: [2, 4, 0, 0, 0, 110, 97, 109, 101] for value name
Encoded TLV: [5, 4, 0, 0, 0, 2, 0, 0, 0] for value 2
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
Column TLV: [99, 0, 0, 0, 24, 0, 0, 0, 2, 4, 0, 0, 0, 110, 97, 109, 101, 5, 4, 0, 0, 0, 2, 0, 0, 0, 4, 1, 0, 0, 0, 0] (len=32)
The value b'c\x00\x00\x00\x18\x00\x00\x00\x02\x04\x00\x00\x00name\x05\x04\x00\x00\x00\x02\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 32 bytes with total data be

OSError: expected 1 bytes, got 0

OSError: "/usr/bin/fish" shell not found